In [2]:
# Homework. Lesson 20
import sqlite3
import pandas as pd

In [ ]:
# 1) Customer Purchases Analysis:
conn = sqlite3.connect('chinook.db')

customers = pd.read_sql("SELECT CustomerId, FirstName, LastName FROM customers", conn)
invoices = pd.read_sql("SELECT InvoiceID, CustomerId, Total FROM invoices", conn)

customer_spent = invoices.groupby('CustomerId')['Total'].sum().reset_index()

result = customer_spent.merge(customers, on='CustomerId')
result['CustomerName'] = result['FirstName'] + ' ' + result['LastName']

top5 = result.sort_values(by='Total', ascending=False).head(5)
print(top5[['CustomerId', 'CustomerName', 'Total']])

    CustomerId        CustomerName  Total
5            6         Helena Holý  49.62
25          26  Richard Cunningham  47.62
56          57          Luis Rojas  46.62
44          45     Ladislav Kovács  45.62
45          46       Hugh O'Reilly  45.62


In [ ]:
# 2) Album vs. Individual Track Purchases:
conn = sqlite3.connect('chinook.db')

invoice_lines = pd.read_sql("SELECT InvoiceLineId, InvoiceId, TrackId FROM invoice_items", conn)
invoices = pd.read_sql("SELECT InvoiceId, CustomerId FROM invoices", conn)
tracks = pd.read_sql("SELECT TrackId, AlbumId FROM tracks", conn)
albums = pd.read_sql("SELECT albumId FROM albums", conn)

data = invoice_lines.merge(invoices, on= 'InvoiceId').merge(tracks, on= 'TrackId')

album_track_counts = tracks.groupby("AlbumId")["TrackId"].nunique().reset_index(name="AlbumTotalTracks")
cust_album_tracks = data.groupby(["CustomerId", "AlbumId"])["TrackId"].nunique().reset_index(name="TracksBought")
cust_album_tracks = cust_album_tracks.merge(album_track_counts, on="AlbumId")
cust_album_tracks["IsFullAlbum"] = cust_album_tracks["TracksBought"] == cust_album_tracks["AlbumTotalTracks"]
customer_pref = cust_album_tracks.groupby("CustomerId")["IsFullAlbum"].max().reset_index()
customer_pref["Category"] = customer_pref["IsFullAlbum"].apply(lambda x: "Full Album" if x else "Individual Tracks")
summary = customer_pref["Category"].value_counts(normalize=True) * 100
print(summary)  

Category
Individual Tracks    64.40678
Full Album           35.59322
Name: proportion, dtype: float64
